## Optimisation

Zoptymalizować program pod kątem wydajności:
- Pobrać przygotowany kod niezoptymalizowanego programu. Jest to prosta biblioteka do reprezentacji grafów, która zawiera funkcje do tworzenia grafów w reprezentacji tablicowej i obiektowej, a także przykłady operacji wykonywanych na grafach (bfs, sprawdzanie cyklu Eulera) i funkcje wypisujące elementy grafu.
- Na podstawie materiałów z ćwiczeń i wykładu znaleźć i poprawić fragmenty, które zostały napisane niezgodnie z zasadami tworzenia wydajnych aplikacji w Julii. 
- Aby dostać maksymalną liczbę punktów, należy wykonać przynajmniej 5 (różnych) optymalizacji.
- W celu weryfikacji poprawek należy użyć narzędzi do pomiaru czasu i alokowanej pamięci. 

Source code:
https://github.com/kzajac/kzajac.github.io/blob/master/julia/lab2/graphs.jl

## Original module

In [1]:
module Graphs

using StatsBase

export GraphVertex, NodeType, Person, Address,
       generate_random_graph, get_random_person, get_random_address, generate_random_nodes,
       convert_to_graph,
       bfs, check_euler, partition,
       graph_to_str, node_to_str,
       test_graph

#= Single graph vertex type.
Holds node value and information about adjacent vertices =#
type GraphVertex
  value
  neighbors ::Vector
end

# Types of valid graph node's values.
abstract NodeType

type Person <: NodeType
  name
end

type Address <: NodeType
  streetNumber
end


# Number of graph nodes.
N = 800

# Number of graph edges.
K = 10000


#= Generates random directed graph of size N with K edges
and returns its adjacency matrix.=#
function generate_random_graph()
    A = Array{Int64,2}(N, N)

    for i=1:N, j=1:N
      A[i,j] = 0
    end

    for i in sample(1:N*N, K, replace=false)
      row, col = ind2sub(size(A), i)
      A[row,col] = 1
      A[col,row] = 1
    end
    A
end

# Generates random person object (with random name).
function get_random_person()
  Person(randstring())
end

# Generates random person object (with random name).
function get_random_address()
  Address(rand(1:100))
end

# Generates N random nodes (of random NodeType).
function generate_random_nodes()
  nodes = Vector()
  for i= 1:N
    push!(nodes, rand() > 0.5 ? get_random_person() : get_random_address())
  end
  nodes
end

#= Converts given adjacency matrix (NxN)
  into list of graph vertices (of type GraphVertex and length N). =#
function convert_to_graph(A, nodes)
  N = length(nodes)
  push!(graph, map(n -> GraphVertex(n, GraphVertex[]), nodes)...)

  for i = 1:N, j = 1:N
      if A[i,j] == 1
        push!(graph[i].neighbors, graph[j])
      end
  end
end

#= Groups graph nodes into connected parts. E.g. if entire graph is connected,
  result list will contain only one part with all nodes. =#
function partition()
  parts = []
  remaining = Set(graph)
  visited = bfs(remaining=remaining)
  push!(parts, Set(visited))

  while !isempty(remaining)
    new_visited = bfs(visited=visited, remaining=remaining)
    push!(parts, new_visited)
  end
  parts
end

#= Performs BFS traversal on the graph and returns list of visited nodes.
  Optionally, BFS can initialized with set of skipped and remaining nodes.
  Start nodes is taken from the set of remaining elements. =#
function bfs(;visited=Set(), remaining=Set(graph))
  first = next(remaining, start(remaining))[1]
  q = [first]
  push!(visited, first)
  delete!(remaining, first)
  local_visited = Set([first])

  while !isempty(q)
    v = pop!(q)

    for n in v.neighbors
      if !(n in visited)
        push!(q, n)
        push!(visited, n)
        push!(local_visited, n)
        delete!(remaining, n)
      end
    end
  end
  local_visited
end

#= Checks if there's Euler cycle in the graph by investigating
   connectivity condition and evaluating if every vertex has even degree =#
function check_euler()
  if length(partition()) == 1
    return all(map(v -> iseven(length(v.neighbors)), graph))
  end
    "Graph is not connected"
end

#= Returns text representation of the graph consisiting of each node's value
   text and number of its neighbors. =#
function graph_to_str()
  graph_str = ""
  for v in graph
    graph_str *= "****\n"

    n = v.value
    if isa(n, Person)
      node_str = "Person: $(n.name)\n"
    else isa(n, Address)
      node_str = "Street nr: $(n.streetNumber)\n"
    end

    graph_str *= node_str
    graph_str *= "Neighbors: $(length(v.neighbors))\n"
  end
  graph_str
end

#= Tests graph functions by creating 100 graphs, checking Euler cycle
  and creating text representation. =#
function test_graph()
  for i=1:100
    global graph = GraphVertex[]

    A = generate_random_graph()
    nodes = generate_random_nodes()
    convert_to_graph(A, nodes)

    str = graph_to_str()
    # println(str)
    # println(check_euler())
    res = check_euler()
  end
end

end

Graphs

In [2]:
using Graphs
@time test_graph()

 14.690596 seconds (125.26 M allocations: 6.894 GB, 6.67% gc time)


In [3]:
using BenchmarkTools
@benchmark test_graph()

BenchmarkTools.Trial: 
  memory estimate:  6.85 GiB
  allocs estimate:  124314103
  --------------
  minimum time:     12.614 s (7.43% GC)
  median time:      12.614 s (7.43% GC)
  mean time:        12.614 s (7.43% GC)
  maximum time:     12.614 s (7.43% GC)
  --------------
  samples:          1
  evals/sample:     1

## Optimised module

In [1]:
module Graphs_Optimised

using StatsBase

export GraphVertex, NodeType, Person, Address,
       generate_random_graph, get_random_person, get_random_address, generate_random_nodes,
       convert_to_graph,
       bfs, check_euler, partition,
       graph_to_str, node_to_str,
       test_graph

abstract NodeType

#= Single graph vertex type.
Holds node value and information about adjacent vertices =#
type GraphVertex
  value::NodeType        # type indication -> optimisation
  neighbors::Vector      # type indication -> optimisation
end

# Types of valid graph node's values.

type Person <: NodeType
    name::String         # type indication -> optimisation
end

type Address <: NodeType
    streetNumber::Int64  # type indication -> optimisation
end

# Number of graph nodes.
const N = 800   # global constant variable -> optimisation

# Number of graph edges.
const K = 10000 # global constant variable -> optimisation


#= Generates random directed graph of size N with K edges
and returns its adjacency matrix.=#

function generate_random_graph()
    A = zeros((N,N))       # using zeros() -> optimisation
    for i in sample(1:N*N, K, replace=false)
      A[i] = 1
    end
    A
end

# Generates random person object (with random name).
function get_random_person()
    Person(randstring()::String) # type indication -> optimisation
end

# Generates random person object (with random name).
function get_random_address()
    Address(rand(1:100)::Int64) # type indication -> optimisation
end

# Generates N random nodes (of random NodeType).
function generate_random_nodes()
  nodes = Vector()
  for i= 1:N
   push!(nodes, rand() > 0.5 ? get_random_person() : get_random_address())
  end
  nodes
end

#= Converts given adjacency matrix (NxN)
  into list of graph vertices (of type GraphVertex and length N). =#
function convert_to_graph(A, nodes)
  graph = GraphVertex[]
  push!(graph, map(n -> GraphVertex(n, GraphVertex[]), nodes)...)

  for j = 1:N, i = 1:N
    if A[i,j] == 1          # iteration change -> optimisation
    push!(graph[j].neighbors, graph[i])
    end
  end
  graph
end

#= Groups graph nodes into connected parts. E.g. if entire graph is connected,
  result list will contain only one part with all nodes. =#
function partition(graph)
  parts = []
  remaining = Set(graph)
  visited = bfs(graph, remaining=remaining)
  push!(parts, Set(visited))

  while !isempty(remaining)
    new_visited = bfs(graph, visited=visited, remaining=remaining)
    push!(parts, new_visited)
  end
  parts
end

#= Performs BFS traversal on the graph and returns list of visited nodes.
  Optionally, BFS can initialized with set of skipped and remaining nodes.
  Start nodes is taken from the set of remaining elements. =#
function bfs(graph::Array{GraphVertex,1}; visited=Set(), remaining=Set(graph))
  first = next(remaining, start(remaining))[1]
  q = [first]
  push!(visited, first)
  delete!(remaining, first)
  local_visited = Set([first])

  while !isempty(q)
    v = pop!(q)

    for n in v.neighbors
      if !(n in visited)
        push!(q, n)
        push!(visited, n)
        push!(local_visited, n)
        delete!(remaining, n)
      end
    end
  end
  local_visited
end

#= Checks if there's Euler cycle in the graph by investigating
   connectivity condition and evaluating if every vertex has even degree =#
function check_euler(graph::Array{GraphVertex,1})
  if length(partition(graph)) == 1
    return all(map(v -> iseven(length(v.neighbors)), graph))
  end
    "Graph is not connected"
end

        
#= Returns text representation of the graph consisiting of each node's value
   text and number of its neighbors. =#
        
node_to_str(p::Person) = "Person: $(p.name)\n"          
node_to_str(a::Address) = "Street nr: $(a.streetNumber)\n"  
neighbors_to_str(v::Vector) = "Neighbors: $(length(v))\n"
        
function graph_vertex_to_str(v::GraphVertex)     # division into smaller functions -> optimisation
    graph_vertex_str = "****\n"
    graph_vertex_str *= node_to_str(v.value)
    graph_vertex_str *= neighbors_to_str(v.neighbors)  # join mozna zrobic
end
        
function graph_to_str(graph::Array{GraphVertex,1})
  graph_str = ""
  for v in graph
    graph_str *= graph_vertex_to_str(v)
  end
  graph_str
end
        
             
#= Tests graph functions by creating 100 graphs, checking Euler cycle
  and creating text representation. =#
function test_graph()
  for i=1:100
    # global graph = GraphVertex[] # passing graph as function argument, not global variable -> optimisation

    A = generate_random_graph()
    nodes = generate_random_nodes()
    graph = convert_to_graph(A, nodes)

    str = graph_to_str(graph)
    #println(str)
    #println(check_euler(graph))
    res = check_euler(graph)
    
  end
end

end

  likely near In[1]:23
  likely near In[1]:23


Graphs_Optimised

In [2]:
using Graphs_Optimised
@time test_graph()

  2.152774 seconds (4.30 M allocations: 1.976 GB, 5.92% gc time)


  likely near In[2]:155
  likely near In[2]:155
  likely near In[2]:155
  likely near In[2]:155
  likely near In[2]:155


In [3]:
using BenchmarkTools
@benchmark test_graph()

BenchmarkTools.Trial: 
  memory estimate:  1.94 GiB
  allocs estimate:  3404562
  --------------
  minimum time:     1.237 s (9.87% GC)
  median time:      1.324 s (9.68% GC)
  mean time:        1.371 s (9.62% GC)
  maximum time:     1.599 s (9.32% GC)
  --------------
  samples:          4
  evals/sample:     1